In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [2]:
# create driver windows
driver = Chrome()
# mac
#driver = Chrome('/Users/christophermena/Downloads/chromedriver')

In [3]:
# navigate to fb
driver.get('https://www.instagram.com/')

In [4]:
username = driver.find_element(By.XPATH, '//input[@name="username"]')

In [5]:
username.send_keys('iamcriss_1')

In [6]:
password = driver.find_element(By.XPATH, '//input[@name="password"]')

In [7]:
password.send_keys('titi020696')

In [8]:
password.send_keys(Keys.RETURN)

In [9]:
save_btm = driver.find_element(By.XPATH, '//div[@class="cmbtv"]').click()

In [10]:
notification_btn = driver.find_element(By.XPATH, '//div[@class="mt3GC"]').click()

In [11]:
search = driver.find_element(By.XPATH, '//div[@class=" cTBqC"]').click()

In [12]:
search = driver.find_element(By.XPATH, '//input[@placeholder="Search"]')

In [13]:
search.send_keys('#nycdot')

In [14]:
search.send_keys(Keys.RETURN)

In [15]:
dotpage = driver.find_element(By.XPATH, '//a[@href="/explore/tags/nycdot/"]').click()

In [16]:
pages = driver.find_element(By.XPATH, '//article[@class="KC1QD"]')

In [17]:
recent_pages = pages.find_element(By.XPATH, './div[2]/div[1]')

In [18]:
cards = recent_pages.find_elements(By.XPATH, './/div[@class="Nnq7C weEfm"]')

In [19]:
len(cards)

8

In [20]:
card = cards[0]

In [21]:
posts = card.find_elements(By.XPATH, './/div[@class="v1Nh3 kIKUG _bz0w"]')

In [22]:
post = posts[0]

In [23]:
post.find_element(By.XPATH, './/a').click()

In [24]:
post_content = post.find_element(By.XPATH, '//div[@role="presentation"]')

In [25]:
username = post_content.find_element(By.XPATH, './/h2[@class="_6lAjh "]').text

In [26]:
comment = post_content.find_element(By.XPATH, './/span[@class="_7UhW9   xLCgt      MMzan   KV-D4            se6yk       T0kll "]').text

In [47]:
time = post_content.find_element(By.XPATH, './/time[@class="_1o9PC"]').get_attribute('datetime')

In [48]:
time = time[:10]

In [43]:
time

'2022 04 19'

In [44]:
import datetime

In [45]:
dt = datetime.datetime.strptime(time, '%Y %m %d')

In [46]:
dt

datetime.datetime(2022, 4, 19, 0, 0)

In [ ]:
# close post
post_content.find_element(By.XPATH, './/button[@class="wpO6b  "]').click()

In [ ]:
def getPostData(post):
    """
    Extract data from instagram data.
    Takes a post card  Selenium WebElement as a parameter
    """
    post.find_element(By.XPATH, './/a').click()
    sleep(3)
    post_content = post.find_element(By.XPATH, '//div[@role="presentation"]')
    try: username = post_content.find_element(By.XPATH, './/h2[@class="_6lAjh "]').text
    except NoSuchElementException:
        return
    comment = post_content.find_element(By.XPATH, './/span[@class="_7UhW9   xLCgt      MMzan   KV-D4            se6yk       T0kll "]').text
    time = post_content.find_element(By.XPATH, './/time[@class="_1o9PC"]').get_attribute('datetime')
    time = time[:10]
    # close post
    post_content.find_element(By.XPATH, './/button[@class="wpO6b  "]').click()
    sleep(1)
    content = (username, comment, time)
    return content


In [ ]:
data = []
post_ids = set()
last_position = driver.execute_script('return window.pageYOffset;')
scrolling = True

In [ ]:
while scrolling:
    # load post
    cards = recent_pages.find_elements(By.XPATH, './/div[@class="Nnq7C weEfm"]')
    for card in cards[-15:]:
         posts = card.find_elements(By.XPATH, './/div[@class="v1Nh3 kIKUG _bz0w"]')
         for post in posts:
             comment = getPostData(post)
             if comment:
                 # create unique id
                 post_id = ''.join(comment)
                 # if id not in post_ids
                 if post_id not in post_ids:
                     post_ids.add(post_id)
                     data.append(comment)
    # scroll attempt
    scroll_attempt = 0
    while True:
        # scroll down page
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') 
        sleep(3)
        current_position = driver.execute_script('return window.pageYOffset;')
        if last_position == current_position:
            scroll_attemtp += 1
            # end scrolling
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(3)
        else:
            last_position = current_position
            break


In [ ]:
for card in cards:
    posts = card.find_elements(By.XPATH, './/div[@class="v1Nh3 kIKUG _bz0w"]')
    for post in posts:
        data.append(getPostData(post))
        sleep(2)

In [ ]:
driver.close()

In [ ]:
# saving data
with open('ig_data.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Comments', 'Time']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)